<a href="https://www.kaggle.com/code/agastyapatel/nlp-emotion-classifier?scriptVersionId=163637967" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv


In [45]:
isKaggle = os.getenv('KAGGLE_KERNEL_RUN_TYPE')

In [46]:
from pathlib import Path

if isKaggle:
    path = Path('../input/emotions/text.csv')

In [47]:
import pandas as pd

In [48]:
df = pd.read_csv(path)
df

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...,...
416804,416804,i feel like telling these horny devils to find...,2
416805,416805,i began to realize that when i was feeling agi...,3
416806,416806,i feel very curious be why previous early dawn...,5
416807,416807,i feel that becuase of the tyranical nature of...,3


In [49]:
vocab = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

# DataIngestion and Sampling


In [50]:
df.describe(include='object')

,text
count,416809
unique,393822
top,i feel more adventurous willing to take risks
freq,16


In [51]:
df = df.drop(columns = ['Unnamed: 0'])

In [52]:
df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [53]:
sample = df.sample(n=1000)
sample

,text,label
36130,id say maybe made them feel foolish but that w...,0
138065,i joined the lds church i admit to feeling som...,0
146440,i must admit i didnt feel like hugging him not...,3
103337,i hate that i can still feel if any nerve is d...,0
315528,im actually feeling a little smug,1
...,...,...
106984,i feel terrible i feel that there should be mo...,0
70108,i see all the review books and requests that o...,3
218447,i am with adults you always have to try and wo...,2
338956,i didnt have a chance to breathe and plus the ...,4


In [54]:
sample.describe(include ='all')

,text,label
count,1000,1000.000000
unique,1000,NaN
top,i started feeling benign raindrops fall on my ...,NaN
freq,1,NaN
mean,NaN,1.553000
std,NaN,1.493795
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,1.000000
75%,NaN,3.000000


In [55]:
sample.columns

Index(['text', 'label'], dtype='object')

In [56]:
sample = sample.rename(columns={'text': 'input'})

In [57]:
sample.iloc[1]

input    i joined the lds church i admit to feeling som...
label                                                    0
Name: 138065, dtype: object

In [58]:
sample.describe(include='all')

,input,label
count,1000,1000.000000
unique,1000,NaN
top,i started feeling benign raindrops fall on my ...,NaN
freq,1,NaN
mean,NaN,1.553000
std,NaN,1.493795
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,1.000000
75%,NaN,3.000000


In [59]:
sample['label'] = sample['label'].astype(float)

In [60]:
sample

,input,label
36130,id say maybe made them feel foolish but that w...,0.0
138065,i joined the lds church i admit to feeling som...,0.0
146440,i must admit i didnt feel like hugging him not...,3.0
103337,i hate that i can still feel if any nerve is d...,0.0
315528,im actually feeling a little smug,1.0
...,...,...
106984,i feel terrible i feel that there should be mo...,0.0
70108,i see all the review books and requests that o...,3.0
218447,i am with adults you always have to try and wo...,2.0
338956,i didnt have a chance to breathe and plus the ...,4.0


# Tokenization
Transformers uses `Dataset` object for storing dataset 

In [61]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(sample)

In [62]:
ds

Dataset({
    features: ['input', 'label', '__index_level_0__'],
    num_rows: 1000
})

In [63]:
model = 'google-bert/bert-base-uncased'

AutoTokenizer will create a tokenizer appropriate for the model

In [64]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertForSequenceClassification
tokz = AutoTokenizer.from_pretrained(model)

In [65]:
tokz

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [66]:
tokz.tokenize('i sometimes feel as though i make up reasons to be mad just because i still have that tasteless feeling growing inside of me')

['i',
 'sometimes',
 'feel',
 'as',
 'though',
 'i',
 'make',
 'up',
 'reasons',
 'to',
 'be',
 'mad',
 'just',
 'because',
 'i',
 'still',
 'have',
 'that',
 'taste',
 '##less',
 'feeling',
 'growing',
 'inside',
 'of',
 'me']

In [67]:
def tok_function(x): return tokz(x['input'])

In [68]:
tok_ds = ds.map(tok_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [69]:
tok_ds

Dataset({
    features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [70]:
tok_ds[0]['input'], tok_ds[0]['input_ids']

('id say maybe made them feel foolish but that would be reeeeeeally reaching',
 [101,
  8909,
  2360,
  2672,
  2081,
  2068,
  2514,
  13219,
  2021,
  2008,
  2052,
  2022,
  2128,
  4402,
  4402,
  15879,
  2135,
  4285,
  102])

In [71]:
# Getting token of `1` 
tokz.vocab['1']

1015

In [72]:
tok_ds.rename_columns({'label': 'labels'})

Dataset({
    features: ['input', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

## Splitting Data

In [73]:
# Transformers uses DatasetDict to store validation and training sets
dds = tok_ds.train_test_split(0.1, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

## Setting Metric

In [74]:
# Correlation Coefficient
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [75]:
#np.corrcoef(sample['input'], sample['label'])

Transformers expects metric to be returned as dict. 

In [76]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

# Training

In [77]:
from transformers import TrainingArguments, Trainer

In [78]:
bs = 128
epoch = 4
lr = 8e-5

In [79]:
# TrainingArgument class is used to set up the arguments
args = TrainingArguments('outputs', learning_rate=lr, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True,
                         evaluation_strategy='epoch',
                         weight_decay=0.01, 
                         report_to='none')

In [80]:
model = AutoModelForSequenceClassification.from_pretrained(model, num_labels =1)
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
"""model = BertForSequenceClassification.from_pretrained(model, num_labels =1)
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz,
                  compute_metrics=corr_d)"""

"model = BertForSequenceClassification.from_pretrained(model, num_labels =1)\ntrainer = Trainer(model, \n                  args, \n                  train_dataset=dds['train'],\n                  eval_dataset=dds['test'],\n                  tokenizer=tokz,\n                  compute_metrics=corr_d)"

In [82]:
trainer.train();

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,1.830285
2,No log,1.142335
3,No log,0.608093


In [83]:
print(dds['train'].features['label'].dtype)
print(dds['test'].features['label'].dtype)

float64
float64


# Prediction

In [84]:
sample

,input,label
36130,id say maybe made them feel foolish but that w...,0.0
138065,i joined the lds church i admit to feeling som...,0.0
146440,i must admit i didnt feel like hugging him not...,3.0
103337,i hate that i can still feel if any nerve is d...,0.0
315528,im actually feeling a little smug,1.0
...,...,...
106984,i feel terrible i feel that there should be mo...,0.0
70108,i see all the review books and requests that o...,3.0
218447,i am with adults you always have to try and wo...,2.0
338956,i didnt have a chance to breathe and plus the ...,4.0


In [119]:
data = {
    'input': [ 'Feeling a deep sense of sorrow overwhelms me.',
                'Experiencing immense happiness fills my heart with warmth.',
                'Overflowing with affection, I cherish every moment.',
                'Anger simmers within me, ready to boil over at any provocation.',
                'oh my god!! you\'re back!',
                 'Aaaahhh! That\'s a big shock!' ]
}
eval_df = pd.DataFrame(data)
eval_df

,input
0,Feeling a deep sense of sorrow overwhelms me.
1,Experiencing immense happiness fills my heart ...
2,"Overflowing with affection, I cherish every mo..."
3,"Anger simmers within me, ready to boil over at..."
4,oh my god!! you're back!
5,Aaaahhh! That's a big shock!


In [120]:
eval_ds = Dataset.from_pandas(eval_df).map(tok_function, batched=True)
eval_ds

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6
})

In [121]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[0.38946426],
       [0.97868097],
       [1.10870337],
       [3.31696415],
       [1.5294764 ],
       [3.25920153]])

In [122]:
preds = np.round(preds)
preds

array([[0.],
       [1.],
       [1.],
       [3.],
       [2.],
       [3.]])

In [123]:
print(vocab)
print(vocab[0])
print(vocab[1])
print(vocab[1])
print(vocab[3])
print(vocab[2])
print(vocab[3])

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
sadness
joy
joy
anger
love
anger
